In [1]:
import os
import pandas as pd
from PIL import Image
from sklearn.metrics import mean_absolute_error, confusion_matrix
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score
from matplotlib import pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
from torchvision.models import resnet18, resnet50, ResNet18_Weights, ResNet50_Weights, ResNet152_Weights
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import glob
import numpy as np
import seaborn as sns
import random

from torch.optim.lr_scheduler import ReduceLROnPlateau

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [2]:
# Set random seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [3]:
# Load CSV file
csv_file = "/home/janz/PROJECT/Tabular_Datas/Tabular_for_cnn_updated.csv"
# df = pd.read_csv(csv_file, nrows=10000)
df = pd.read_csv(csv_file)

# Scale the target variable by 100
df['Last_UCVA'] = df['Last_UCVA'] * 10
# df['Last_Efficacy Index'] = df['Last_Efficacy Index'] * 10

# Define directories
right_eye_dir = "/home/janz/PROJECT/Eye_Scans_Data/op2_png"
left_eye_dir = "/home/janz/PROJECT/Eye_Scans_Data/op5_png"

# Helper function to load and concatenate images
def load_images(ocular_treatment_id):
    id_code, eye, _, _ = ocular_treatment_id.split('-')
    eye_dir = right_eye_dir if eye == 'Right' else left_eye_dir
    anterior_image_path = glob.glob(os.path.join(eye_dir, f"{id_code}_*_Tangential_anterior.png"))[0]
    posterior_image_path = glob.glob(os.path.join(eye_dir, f"{id_code}_*_Tangential_posterior.png"))[0]
    anterior_image = Image.open(anterior_image_path).convert('RGB')
    posterior_image = Image.open(posterior_image_path).convert('RGB')
    return anterior_image, posterior_image

# Custom Dataset
class EyeDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe  # CSV file as a dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        ocular_treatment_id = self.dataframe.iloc[idx]['Ocular Treatment ID']
        last_ucva = self.dataframe.iloc[idx]['Last_UCVA']
        # last_ucva = self.dataframe.iloc[idx]['Last_Efficacy Index']
        anterior_image, posterior_image = load_images(ocular_treatment_id)
        
        # Concatenate images side by side (width-wise)
        combined_image = Image.new('RGB', (anterior_image.width + posterior_image.width, anterior_image.height))
        combined_image.paste(anterior_image, (0, 0))
        combined_image.paste(posterior_image, (anterior_image.width, 0))
        
        if self.transform:
            combined_image = self.transform(combined_image)

        return combined_image, torch.tensor(last_ucva, dtype=torch.float)

# Create dataset and dataloaders
transform = transforms.Compose([
    # transforms.Resize((800, 290)),  # Resizing to maintain aspect ratio of concatenated images
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


dataset = EyeDataset(df, transform=transform)

# Set seed before splitting the dataset
set_seed(42)

# Split dataset
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

val_size = int(0.2 * train_size)
train_size = train_size - val_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

In [7]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score
from tabulate import tabulate# Define different threshold configurations for evaluation

set_seed(42)

class ResNetModel(nn.Module):
    def __init__(self):
        super(ResNetModel, self).__init__()
        self.resnet = resnet50(weights=ResNet50_Weights.DEFAULT)  # Load pretrained ResNet-50
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 1)  # Modify final layer for regression (1 output)
        self.feature_extractor = nn.Linear(self.resnet.fc.in_features, 15)  # Feature extraction layer

    def forward(self, x):
        # Pass input through ResNet layers individually
        x = self.resnet.conv1(x)
        x = self.resnet.bn1(x)
        x = self.resnet.relu(x)
        x = self.resnet.maxpool(x)

        x = self.resnet.layer1(x)
        x = self.resnet.layer2(x)
        x = self.resnet.layer3(x)
        x = self.resnet.layer4(x)

        # Apply global average pooling (or other pooling method)
        x = self.resnet.avgpool(x)
        x = torch.flatten(x, 1)  # Flatten to 2D (batch_size, 2048)

        # Feature extraction layer
        features = self.feature_extractor(x)
        # Final output
        output = self.resnet.fc(x)

        return features, output
    
# Function for training the model
def train_model(config):
    # Unpack configuration
    num_epochs = config['num_epochs']
    batch_size = config['batch_size']
    learning_rate = config['learning_rate']

    model = ResNetModel().to(device)

    # Wrap the model with DataParallel
    if torch.cuda.device_count() > 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(model)

    # Loss and optimizer
    criterion = nn.MSELoss()
    # criterion = nn.L1Loss()
    # criterion = nn.SmoothL1Loss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0)
    # Learning rate scheduler
    scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.1)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=32)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=32)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=32)

    best_model_wts = model.state_dict()
    best_loss = float('inf')
    best_mae = float('inf')

    train_losses = []  # To store training losses
    val_losses = []    # To store validation losses

    # Early stopping
    best_val_loss = float('inf')
    patience = 4
    epochs_without_improvement = 0
    #####

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
                dataloader = train_loader
            else:
                model.eval()
                dataloader = val_loader

            running_loss = 0.0
            running_mae = 0.0

            # Iterate over data
            for inputs, targets in tqdm(dataloader):
                inputs = inputs.to(device)
                targets = targets.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                with torch.set_grad_enabled(phase == 'train'):
                    # outputs = model(inputs)
                    features, outputs = model(inputs)
                    loss = criterion(outputs, targets.unsqueeze(1))
                    mae = mean_absolute_error(targets.detach().cpu().numpy(), outputs.detach().cpu().numpy())

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_mae += mae * inputs.size(0)

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_mae = running_mae / len(dataloader.dataset)

            print(f'{phase} Loss: {epoch_loss:.4f}, MAE: {epoch_mae:.4f}')

            # Store losses
            if phase == 'train':
                train_losses.append(epoch_loss)
            else:
                val_losses.append(epoch_loss)

            # Deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_mae = epoch_mae
                best_model_wts = model.state_dict()
        
        # Step the scheduler
        scheduler.step(epoch_loss)

        # Early stopping
        if phase == 'val' and epoch_loss < best_val_loss:
            best_val_loss = epoch_loss
            epochs_without_improvement = 0
            # Save best model weights

        else:
            epochs_without_improvement += 1
            if epochs_without_improvement > patience:
                print(f'Early stopping at epoch {epoch}')
                break

    return model, {
        'train_loss': train_losses[-1],
        'val_loss': val_losses[-1]
    }

def evaluate_model(model, config):
    test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, num_workers=32)

    with torch.no_grad():
        total_mae = 0.0
        total_mse = 0.0
        all_predictions = []
        all_targets = []
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            _, outputs = model(inputs)
            total_mae += torch.mean(torch.abs(outputs - targets)).item()
            total_mse += torch.mean((outputs - targets) ** 2).item()
            all_predictions.extend(outputs.cpu().numpy().flatten())
            all_targets.extend(targets.cpu().numpy().flatten())
        mae = total_mae / len(test_loader)
        mse = total_mse / len(test_loader)

    all_predictions = np.array(all_predictions)
    all_targets = np.array(all_targets)
    abs_difference = np.abs(all_predictions - all_targets)
    abs_difference_mask = abs_difference <= config['abs_difference_threshold']

    binary_predictions = np.where(all_predictions >= config['binary_classification_threshold'], 1, 0)
    binary_targets = np.where(all_targets >= config['binary_classification_threshold'], 1, 0)
    for i in range(len(binary_predictions)):
        if abs_difference_mask[i]:
            binary_predictions[i] = binary_targets[i]

    precision = precision_score(binary_targets, binary_predictions)
    recall = recall_score(binary_targets, binary_predictions)
    accuracy = accuracy_score(binary_targets, binary_predictions)
    auc = roc_auc_score(binary_targets, binary_predictions)
    true_negatives = np.sum((binary_targets == 0) & (binary_predictions == 0))
    false_positives = np.sum((binary_targets == 0) & (binary_predictions == 1))
    false_alarm_rate = false_positives / (false_positives + true_negatives)

    return {
        'mae': mae,
        'mse': mse,
        'precision': precision,
        'recall': recall,
        'accuracy': accuracy,
        'auc': auc,
        'false_alarm_rate': false_alarm_rate,
        'config': config  # Include configuration in the results
    }

# Define the training configuration
training_config = {
    'num_epochs': 10,
    'batch_size': 32,
    'learning_rate': 0.00001,
}

# Train the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Assuming X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, X_test_tensor, y_test_tensor are already defined
trained_model, training_results = train_model(training_config)
print(f"Training results: {training_results}")

Let's use 3 GPUs!
Epoch 0/9
----------


100%|██████████| 1071/1071 [04:24<00:00,  4.04it/s]


train Loss: 10.6944, MAE: 2.1538


100%|██████████| 268/268 [00:54<00:00,  4.93it/s]


val Loss: 3.1105, MAE: 1.1600
Epoch 1/9
----------


...

val Loss: 2.9417, MAE: 1.1559
Epoch 9/9
----------


100%|██████████| 1071/1071 [04:23<00:00,  4.06it/s]


train Loss: 2.8358, MAE: 1.1363


100%|██████████| 268/268 [00:54<00:00,  4.89it/s]

val Loss: 2.9426, MAE: 1.1306
Training results: {'train_loss': 2.8358494448511435, 'val_loss': 2.9425842226728243}


In [8]:
# Define different threshold configurations for evaluation
threshold_configs = [
    {
        'binary_classification_threshold': 9.25,
        'abs_difference_threshold': 0,
        'batch_size': training_config['batch_size'],
        'num_epochs': training_config['num_epochs'],
        'learning_rate': training_config['learning_rate']
    },
    {
        'binary_classification_threshold': 9.25,
        'abs_difference_threshold': 1,
        'batch_size': training_config['batch_size'],
        'num_epochs': training_config['num_epochs'],
        'learning_rate': training_config['learning_rate']
    },
    {
        'binary_classification_threshold': 9,
        'abs_difference_threshold': 0,
        'batch_size': training_config['batch_size'],
        'num_epochs': training_config['num_epochs'],
        'learning_rate': training_config['learning_rate']
    },
    {
        'binary_classification_threshold': 9,
        'abs_difference_threshold': 1,
        'batch_size': training_config['batch_size'],
        'num_epochs': training_config['num_epochs'],
        'learning_rate': training_config['learning_rate']
    },
    {
        'binary_classification_threshold': 8.5,
        'abs_difference_threshold': 0,
        'batch_size': training_config['batch_size'],
        'num_epochs': training_config['num_epochs'],
        'learning_rate': training_config['learning_rate']
    },
    {
        'binary_classification_threshold': 8.5,
        'abs_difference_threshold': 1,
        'batch_size': training_config['batch_size'],
        'num_epochs': training_config['num_epochs'],
        'learning_rate': training_config['learning_rate']
    },
    {
        'binary_classification_threshold': 8,
        'abs_difference_threshold': 0,
        'batch_size': training_config['batch_size'],
        'num_epochs': training_config['num_epochs'],
        'learning_rate': training_config['learning_rate']
    },
    {
        'binary_classification_threshold': 8,
        'abs_difference_threshold': 1,
        'batch_size': training_config['batch_size'],
        'num_epochs': training_config['num_epochs'],
        'learning_rate': training_config['learning_rate']
    }
]

# threshold_configs = [
#     {
#         'binary_classification_threshold': 9.25,
#         'abs_difference_threshold': 0,
#         'batch_size': training_config['batch_size']
#     },
#     {
#         'binary_classification_threshold': 9.25,
#         'abs_difference_threshold': 1,
#         'batch_size': training_config['batch_size']
#     },
#     {
#         'binary_classification_threshold': 9,
#         'abs_difference_threshold': 0,
#         'batch_size': training_config['batch_size']
#     },
#     {
#         'binary_classification_threshold': 9,
#         'abs_difference_threshold': 1,
#         'batch_size': training_config['batch_size']
#     },
#     {
#         'binary_classification_threshold': 8.5,
#         'abs_difference_threshold': 0,
#         'batch_size': training_config['batch_size']
#     },
#     {
#         'binary_classification_threshold': 8.5,
#         'abs_difference_threshold': 1,
#         'batch_size': training_config['batch_size']
#     },
#     {
#         'binary_classification_threshold': 8,
#         'abs_difference_threshold': 0,
#         'batch_size': training_config['batch_size']
#     },
#     {
#         'binary_classification_threshold': 8,
#         'abs_difference_threshold': 1,
#         'batch_size': training_config['batch_size']
#     }
# ]

# Evaluate the model with different thresholds
evaluation_results = []
for config in threshold_configs:
    results = evaluate_model(trained_model, config)
    evaluation_results.append(results)

# Convert results to DataFrame and print as table
results_df = pd.DataFrame(evaluation_results)
# Print configurations as well
results_df['config'] = results_df['config'].apply(lambda x: str(x))
print(tabulate(results_df, headers='keys', tablefmt='psql'))

+----+---------+---------+-------------+----------+------------+----------+--------------------+--------------------------------------------------------------------------------------------------------------------------------------+
|    |     mae |     mse |   precision |   recall |   accuracy |      auc |   false_alarm_rate | config                                                                                                                               |
|----+---------+---------+-------------+----------+------------+----------+--------------------+--------------------------------------------------------------------------------------------------------------------------------------|
|  0 | 1.22449 | 3.34609 |    0.726197 | 0.561278 |   0.568158 | 0.571518 |           0.418242 | {'binary_classification_threshold': 9.25, 'abs_difference_threshold': 0, 'batch_size': 32, 'num_epochs': 10, 'learning_rate': 1e-05} |
|  1 | 1.22229 | 3.34211 |    0.84392  | 0.764598 |   0.74979  | 0.74256

In [ ]:
num_epochs = 25
batch_size = 128
learning_rate = 0.001

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=32)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=32)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=32)

In [ ]:
set_seed(42)

# resnet

# # model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
# model = models.resnet50(weights=ResNet50_Weights.DEFAULT)
# # model = models.resnet152(weights=ResNet152_Weights.IMAGENET1K_V2)

# model.fc = nn.Linear(model.fc.in_features, 1)

# densenet
# model = models.densenet121(weights='IMAGENET1K_V1')
# model.classifier = nn.Linear(model.classifier.in_features, 1)

# Define the ResNet model
class ResNetModel(nn.Module):
    def __init__(self):
        super(ResNetModel, self).__init__()
        self.resnet = resnet50(weights=ResNet50_Weights.DEFAULT)  # Load pretrained ResNet-50
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 1)  # Modify final layer for regression (1 output)
        self.feature_extractor = nn.Linear(self.resnet.fc.in_features, 15)  # Feature extraction layer

    def forward(self, x):
        # Pass input through ResNet layers individually
        x = self.resnet.conv1(x)
        x = self.resnet.bn1(x)
        x = self.resnet.relu(x)
        x = self.resnet.maxpool(x)

        x = self.resnet.layer1(x)
        x = self.resnet.layer2(x)
        x = self.resnet.layer3(x)
        x = self.resnet.layer4(x)

        # Apply global average pooling (or other pooling method)
        x = self.resnet.avgpool(x)
        x = torch.flatten(x, 1)  # Flatten to 2D (batch_size, 2048)

        # Feature extraction layer
        features = self.feature_extractor(x)
        # Final output
        output = self.resnet.fc(x)

        return features, output

model = ResNetModel().to(device)

# Wrap the model with DataParallel
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)

# Loss and optimizer
criterion = nn.MSELoss()
# criterion = nn.L1Loss()
# criterion = nn.SmoothL1Loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0)
# Learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.1)

# Training loop with loss storage and mean absolute error calculation
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=25):
    best_model_wts = model.state_dict()
    best_loss = float('inf')
    best_mae = float('inf')

    train_losses = []  # To store training losses
    val_losses = []    # To store validation losses

    # Early stopping
    best_val_loss = float('inf')
    patience = 6
    epochs_without_improvement = 0
    #####

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
                dataloader = train_loader
            else:
                model.eval()
                dataloader = val_loader

            running_loss = 0.0
            running_mae = 0.0

            # Iterate over data
            for inputs, targets in tqdm(dataloader):
                inputs = inputs.to(device)
                targets = targets.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                with torch.set_grad_enabled(phase == 'train'):
                    # outputs = model(inputs)
                    features, outputs = model(inputs)
                    loss = criterion(outputs, targets.unsqueeze(1))
                    mae = mean_absolute_error(targets.detach().cpu().numpy(), outputs.detach().cpu().numpy())

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_mae += mae * inputs.size(0)

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_mae = running_mae / len(dataloader.dataset)

            print(f'{phase} Loss: {epoch_loss:.4f}, MAE: {epoch_mae:.4f}')

            # Store losses
            if phase == 'train':
                train_losses.append(epoch_loss)
            else:
                val_losses.append(epoch_loss)

            # Deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_mae = epoch_mae
                best_model_wts = model.state_dict()
        
        # Step the scheduler
        scheduler.step(epoch_loss)

        # Early stopping
        if phase == 'val' and epoch_loss < best_val_loss:
            best_val_loss = epoch_loss
            epochs_without_improvement = 0
            # Save best model weights

        else:
            epochs_without_improvement += 1
            if epochs_without_improvement > patience:
                print(f'Early stopping at epoch {epoch}')
                break
        #####

    print(f'Best val Loss: {best_loss:.4f}, Best val MAE: {best_mae:.4f}')

    # Load best model weights
    model.load_state_dict(best_model_wts)

    return model, train_losses, val_losses

# Set seed before training
set_seed(42)

# Train the model
model, train_losses, val_losses = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=num_epochs)

In [ ]:
# import optuna
# from torch.optim.lr_scheduler import StepLR

# def objective(trial):
#     # Define hyperparameters to be optimized
#     lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
#     batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
#     optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'SGD'])
#     weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-2)
#     # step_size = trial.suggest_int('step_size', 1, 10)
#     # gamma = trial.suggest_float('gamma', 0.1, 0.9)
    
#     # Create dataset and dataloaders with the suggested batch size
#     train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=62)
#     val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=62)
    
#     # Load pre-trained DenseNet model and modify the final layer
#     model = models.densenet121(weights='IMAGENET1K_V1')
#     model.classifier = nn.Linear(model.classifier.in_features, 1)
#     model = model.to(device)
    
#     # Define the optimizer
#     if optimizer_name == 'Adam':
#         optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
#     else:
#         optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay, momentum=0.9)
    
#     # Define the learning rate scheduler
#     # scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)
    
#     # Training loop
#     num_epochs = 3  # You can make this a hyperparameter as well if needed
#     for epoch in range(num_epochs):
#         model.train()
#         running_loss = 0.0
#         for inputs, targets in train_loader:
#             inputs = inputs.to(device)
#             targets = targets.to(device)
            
#             optimizer.zero_grad()
#             outputs = model(inputs)
#             loss = criterion(outputs, targets.unsqueeze(1))
#             loss.backward()
#             optimizer.step()
        
#         # Validation step
#         model.eval()
#         val_loss = 0.0
#         with torch.no_grad():
#             for inputs, targets in val_loader:
#                 inputs = inputs.to(device)
#                 targets = targets.to(device)
#                 outputs = model(inputs)
#                 loss = criterion(outputs, targets.unsqueeze(1))
#                 val_loss += loss.item() * inputs.size(0)
        
#         val_loss /= len(val_loader.dataset)
#         # scheduler.step()

#         trial.report(val_loss, epoch)
        
#         if trial.should_prune():
#             raise optuna.exceptions.TrialPruned()
    
#     return val_loss

# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=3)

# # Get the best hyperparameters
# print('Best trial:', study.best_trial.params)


Print the losses curves

In [ ]:
num_epochs = len(train_losses)  # Update to reflect the actual number of epochs trained, useful if early stopping was triggered

def smooth_curve(points, factor=0.8):
    """Smooths the curve for better visualization."""
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points

# Optionally smooth the loss curves
train_losses_smooth = smooth_curve(train_losses)
val_losses_smooth = smooth_curve(val_losses)

plt.figure(figsize=(10, 5))  # Set the figure size for better readability
plt.plot(range(1, num_epochs + 1), train_losses_smooth, label='Training Loss')
plt.plot(range(1, num_epochs + 1), val_losses_smooth, label='Validation Loss')
plt.title('Training and Validation Loss Curves')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)  # Add grid
plt.legend()
plt.show()

Calculate test losses

In [ ]:
# Evaluate the model on the test set
def evaluate_model(model, test_loader):
    model.eval()
    test_loss = 0.0
    test_mae = 0.0
    with torch.no_grad(), tqdm(total=len(test_loader), desc='Testing') as pbar:
        for inputs, targets in test_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            # outputs = model(inputs)
            features, outputs = model(inputs)
            loss = criterion(outputs, targets.unsqueeze(1))
            mae = mean_absolute_error(targets.cpu().numpy(), outputs.cpu().numpy())
            test_loss += loss.item() * inputs.size(0)
            test_mae += mae * inputs.size(0)
            pbar.update(1)  # Update the progress bar
    test_loss /= len(test_loader.dataset)
    test_mae /= len(test_loader.dataset)
    print(f'Test Loss: {test_loss:.4f}, Test MAE: {test_mae:.4f}')

# Evaluate the model on the test set
evaluate_model(model, test_loader)

Evaluate on test dataset

In [ ]:
# Evaluate the model on the test set
def evaluate_model_classification(model, test_loader):
    model.eval()
    all_predictions = []
    all_targets = []

    with torch.no_grad(), tqdm(total=len(test_loader), desc='Testing') as pbar:
        for inputs, targets in test_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            # outputs = model(inputs).squeeze(1)
            features, outputs = model(inputs)
            outputs = outputs.squeeze(1)

            all_predictions.extend(outputs.cpu().numpy().flatten())
            all_targets.extend(targets.cpu().numpy().flatten())
            pbar.update(1)  # Update the progress bar

    all_predictions = np.array(all_predictions)
    all_targets = np.array(all_targets)

    return all_predictions, all_targets


# Evaluate the model
all_predictions, all_targets = evaluate_model_classification(model, test_loader)

Calculate classification methods on the test dataset.

In [ ]:
binary_classification_threshold = 8.5
abs_difference_threshold = 1

# Check if the absolute difference is smaller than the threshold
abs_difference = np.abs(all_predictions - all_targets)
abs_difference_mask = abs_difference <= abs_difference_threshold

# Convert to binary classification based on threshold
binary_predictions = np.where(all_predictions >= binary_classification_threshold, 1, 0)
binary_targets = np.where(all_targets >= binary_classification_threshold, 1, 0)

for i in range(len(binary_predictions)):
    if abs_difference_mask[i]:
        binary_predictions[i] = binary_targets[i]

# Calculate precision, recall, accuracy, and AUC
precision = precision_score(binary_targets, binary_predictions)
recall = recall_score(binary_targets, binary_predictions)
accuracy = accuracy_score(binary_targets, binary_predictions)
auc = roc_auc_score(binary_targets, binary_predictions)

print(f'Test Precision: {precision:.4f}, Recall: {recall:.4f}, Accuracy: {accuracy:.4f}, AUC: {auc:.4f}')

# Calculate false alarm rate (FPR)
true_negatives = np.sum((binary_targets == 0) & (binary_predictions == 0))
false_positives = np.sum((binary_targets == 0) & (binary_predictions == 1))
false_alarm_rate = false_positives / (false_positives + true_negatives)

print(f'False Alarm Rate: {false_alarm_rate:.4f}')

In [ ]:
# Save the trained model
model_save_path = "/home/janz/PROJECT/trained_models/cnn_model.pth"  # You can also use .pt extension
torch.save(model.module.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

In [ ]:
print(all_predictions[:20])
print(all_targets[:20])

In [ ]:
# Get the number of samples in the test dataset
num_test_samples = len(test_dataset)
print(f'Number of samples in the test dataset: {num_test_samples}')


Confusion matrix for the test dataset

In [ ]:
cm = confusion_matrix(binary_targets, binary_predictions)
print("Confusion Matrix:")
print(cm)

# Plot the confusion matrix using seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Plotting the histogram
plt.figure(figsize=(5, 3))
plt.hist(all_predictions, bins=50, edgecolor='k', alpha=0.7)
plt.title('Distribution of Predictions')
plt.xlabel('Predicted Value')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

# Plotting the histogram
plt.figure(figsize=(5, 3))
plt.hist(all_targets, bins=50, edgecolor='k', alpha=0.7)
plt.title('Distribution of actual Targets')
plt.xlabel('Predicted Value')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
thresh = 10
above = 0
lower = 0
# for i in df["Last_Efficacy Index"]:
for i in all_targets:
    if i >= thresh:
        above += 1
    else:
        lower += 1
print(f"There is {above} samples which there target value are above the value of {thresh}")
print(f"There is {lower} samples which there target value are under the value of {thresh}")